<a href="https://colab.research.google.com/github/neennera/AI_pitch/blob/main/Pre_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset

import pandas as pd
import torchaudio
import pickle
import numpy as np

# DataLoader & Base Model

In [2]:
import zipfile
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
'''!pip install patool
!pip install pyunpack
from pyunpack import Archive
Archive("/content/gdrive/MyDrive/AI/processed.rar").extractall("/content/gdrive/MyDrive/AI/DATA")
'''

In [41]:
train_feat_path="/content/gdrive/MyDrive/AI/DATA/processed/test/"
test_feat_path="/content/gdrive/MyDrive/AI/DATA/processed/train/"

train_name=os.listdir(train_feat_path)
test_name=os.listdir(test_feat_path)

class CustomTrainDataset(Dataset):
    def __init__(self, feat_path=train_feat_path, transform=None, target_transform=None):
        self.img_labels = os.listdir(train_feat_path)
        self.feat_path = feat_path 

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        data = (torch.from_numpy(np.load(self.feat_path +self.img_labels[idx])))
        data = data[0]
        label = int(self.img_labels[idx][0:2])
        return data, label

class CustomTestDataset(Dataset):
    def __init__(self, feat_path=test_feat_path , transform=None, target_transform=None):
        self.img_labels = os.listdir(train_feat_path)
        self.feat_path = feat_path 

    def __len__(self):
      return len(self.img_labels)

    def __getitem__(self, idx):
        data = (torch.from_numpy(np.load(self.feat_path +self.img_labels[idx])))
        data = data[0]
        label = int(self.img_labels[idx][0:2])
        return data, label

train_dataset=CustomTrainDataset()
test_dataset=CustomTrainDataset()

In [ ]:
batch_size = 128
epochs = 10
lr = 0.001
device = 'cuda' if torch.cuda.is_available() else 'cpu'

  train_dataset, test_dataset = download_mnist()
  train_dataloader=DataLoader(train_dataset,batch_size=batch_size)
  model = FeedForwardNet().to(device)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



# Train model [MNIST]

In [ ]:
class NeFeedForwardt(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten() #แปลง array 2D ให้เป็น 1D
    self.dense_layer = nn.Sequential(  #การ pack layer เข้าด้วยกัน
        nn.Linear(28*28,256),  #784 -> 256
        nn.ReLU(),
        nn.Linear(256,10)
    )
    self.softmax = nn.Softmax(dim=1) #จำแนกการทาย class เป็น % 10 class [รวมกันได้ 1]
  
  def forward(self, x):
    x = self.flatten(x)
    x = self.dense_layer(x)
    x = self.softmax(x)
    return x

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
def train_one_epoch(model, data_loader, loss_fn, optimiser, device):
  for inputs, targets in data_loader :
    inputs, targets = inputs.to(device), targets.to(device)
  
  #calculate loss
  pred = model(inputs)
  loss = loss_fn(pred, targets)
  #acc = Acc_fn(pred, targets)

  #backpropagate + update weight
  optimiser.zero_grad() #ลบ gradiun จาก batch ก่อนๆ
  loss.backward()
  optimiser.step()

  print(f"Loss : {loss.item()}") #\t Acc:{acc}")

def train(model, data_loader, loss_fn, optimiser, device, epoch):
  for i in range(epoch):
    print(f"Epoch : {i+1}")
    train_one_epoch(model, data_loader, loss_fn, optimiser, device)
    print("---------------------------")
  print("Finished training")

In [ ]:
train(model, train_dataloader, loss_fn, optimiser, device, epochs)
torch.save(model.state_dict(), "model.pth")
print("Trained feed forward net saved at feedforwardnet.pth")

Epoch : 1
Loss : 2.3032054901123047
---------------------------
Epoch : 2
Loss : 2.2902708053588867
---------------------------
Epoch : 3
Loss : 2.276273012161255
---------------------------
Epoch : 4
Loss : 2.260176658630371
---------------------------
Epoch : 5
Loss : 2.241004467010498
---------------------------
Epoch : 6
Loss : 2.21809458732605
---------------------------
Epoch : 7
Loss : 2.1910974979400635
---------------------------
Epoch : 8
Loss : 2.160165309906006
---------------------------
Epoch : 9
Loss : 2.126034736633301
---------------------------
Epoch : 10
Loss : 2.0899498462677
---------------------------
Finished training
Trained feed forward net saved at feedforwardnet.pth


# Predict [MNIST]



In [ ]:
def predict(model, input, target, class_mapping) :
  model.eval() #เป็นสวิตท์ model.eval() ใช่ในการทาย // model.train() ใช้ในการเทรน
  
  with torch.no_grad(): #ใช้ model แบบไม่คำนวน grad
    pred = model(input)
    #print(pred)
    #จะ map ไปเป็น class โดย pred อยู่ในรูป (batch, number , class) -> (1,1,10)
    pred_index = pred[0].argmax(0)
        # pred[0] เอามาแค่ตัวแรก
        # argmax(0)  เลือกเอาอากิวเมนต์ที่มีค่าสูงสุด [เพราะ softmax ทำนายออกมาเป็น % ความน่าจะเป็น]
    pred = class_mapping[pred_index]
    expected = class_mapping[target]
 
  return pred, expected

In [ ]:
class_mapping = [
    "0","1","2","3","4","5","6","7","8","9"
]

if __name__ == "__main__" :
  _ , validation_data = download_mnist()
  #validation_data = ดาต้าที่ใช้ตรวจสอบภความถูกต้อง
  
  model = FeedForwardNet()
  state_dict = torch.load("model.pth")
  model.load_state_dict(state_dict)
  
  for i in range(10) :
    input, target = validation_data[i][0], validation_data[i][1]
    predicted, expected = predict(model, input, target, class_mapping)
    print(f"Predicted: '{predicted}', expected: '{expected}'")

Predicted: '7', expected: '7'
Predicted: '3', expected: '2'
Predicted: '3', expected: '1'
Predicted: '6', expected: '0'
Predicted: '3', expected: '4'
Predicted: '8', expected: '1'
Predicted: '3', expected: '4'
Predicted: '3', expected: '9'
Predicted: '6', expected: '5'
Predicted: '7', expected: '9'


# Conv Model [Urban]

In [ ]:
import zipfile
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
class UrbanSoundDataset(Dataset):
    def __init__(self,
                 annotations_file,
                 audio_dir,
                 transformation,
                 target_sample_rate,
                 num_samples,
                 device):
        self.annotations = pd.read_csv(annotations_file)
        self.audio_dir = audio_dir
        self.device = device
        self.transformation = transformation.to(self.device)
        self.target_sample_rate = target_sample_rate
        self.num_samples = num_samples

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        audio_sample_path = self._get_audio_sample_path(index)
        label = self._get_audio_sample_label(index)
        signal, sr = torchaudio.load(audio_sample_path)
        signal = signal.to(self.device)
        signal = self._resample_if_necessary(signal, sr)
        signal = self._mix_down_if_necessary(signal)
        signal = self._cut_if_necessary(signal)
        signal = self._right_pad_if_necessary(signal)
        signal = self.transformation(signal)
        return signal, label

    def _cut_if_necessary(self, signal):
        if signal.shape[1] > self.num_samples:
            signal = signal[:, :self.num_samples]
        return signal

    def _right_pad_if_necessary(self, signal):
        length_signal = signal.shape[1]
        if length_signal < self.num_samples:
            num_missing_samples = self.num_samples - length_signal
            last_dim_padding = (0, num_missing_samples)
            signal = torch.nn.functional.pad(signal, last_dim_padding)
        return signal

    def _resample_if_necessary(self, signal, sr):
        if sr != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
            signal = resampler(signal)
        return signal

    #อาจใช้ load mono=True มาเลย ให้มันเป็น mono มาตั้งแต่แรก
    def _mix_down_if_necessary(self, signal):
        if signal.shape[0] > 1:
            signal = torch.mean(signal, dim=0, keepdim=True)
        return signal

    def _get_audio_sample_path(self, index):
        fold = f"fold{self.annotations.iloc[index, 5]}"
        path = os.path.join(self.audio_dir, fold, self.annotations.iloc[
            index, 0])
        return path

    def _get_audio_sample_label(self, index):
        return self.annotations.iloc[index, 6]

In [ ]:
#การ clean data ผ่าน cmd
#preprocess ก่อนแล้ว save เพื่อเวลาเทรนจะได้ไวกว่า
ffmpeg -i {ชื่อไฟล์} -ar 22,050 -ac 1
22500 = sample rate
1 = mono / 2 = stereo

In [ ]:
if __name__ == "__main__":
    ANNOTATIONS_FILE = "/content/gdrive/MyDrive/AI/UrbanSound8K/metadata/UrbanSound8K.csv"
    AUDIO_DIR = "/content/gdrive/MyDrive/AI/UrbanSound8K/audio"
    SAMPLE_RATE = 22050
    NUM_SAMPLES = 22050

    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    print(f"Using device {device}")

    mel_spectrogram = torchaudio.transforms.MelSpectrogram(
        sample_rate=SAMPLE_RATE,
        n_fft=1024,
        hop_length=512,
        n_mels=64
    )

    usd = UrbanSoundDataset(ANNOTATIONS_FILE,
                            AUDIO_DIR,
                            mel_spectrogram,
                            SAMPLE_RATE,
                            NUM_SAMPLES,
                            device)
    print(f"There are {len(usd)} samples in the dataset.")
    signal, label = usd[10]

Using device cpu
There are 8732 samples in the dataset.


In [ ]:
for i in range(15):
  signal, label = usd[i]

In [ ]:
class CNNNetwork(nn.Module): #บอกว่าใช้ nn.Module เพื่อมาใช้เป็น template
    def __init__(self):
        super().__init__() #เน้นๆๆ เราต้องใช้ super().__init__() เพื่อก๊อป template จาก nn.module มาใช้ 
        # 4 conv blocks / flatten / linear / softmax
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(128 * 5 * 4, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_data):
        x = self.conv1(input_data)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.flatten(x)
        logits = self.linear(x)
        predictions = self.softmax(logits)
        return x

In [ ]:
from torchsummary import summary
if __name__ == "__main__": # เอาไว้ check งาน library ทำงานถูกต้อง ในไฟล์ที่เราสร้างเป็น function ของ python
    #เช่น เราเก็บฟังก์ชั่นใน Process.py เวลาเรา import Process มันจะมาแค่ฟังก์ชั่น 
    cnn = CNNNetwork()
    summary(cnn, (1, 64, 44))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 66, 46]             160
              ReLU-2           [-1, 16, 66, 46]               0
         MaxPool2d-3           [-1, 16, 33, 23]               0
            Conv2d-4           [-1, 32, 35, 25]           4,640
              ReLU-5           [-1, 32, 35, 25]               0
         MaxPool2d-6           [-1, 32, 17, 12]               0
            Conv2d-7           [-1, 64, 19, 14]          18,496
              ReLU-8           [-1, 64, 19, 14]               0
         MaxPool2d-9             [-1, 64, 9, 7]               0
           Conv2d-10           [-1, 128, 11, 9]          73,856
             ReLU-11           [-1, 128, 11, 9]               0
        MaxPool2d-12            [-1, 128, 5, 4]               0
          Flatten-13                 [-1, 2560]               0
           Linear-14                   

In [ ]:
model=CNNNetwork().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Training Loop

In [ ]:
checkpoint = { 
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
}

def save(train_losses,metrics,baseline,i):
    loss_log=[]
    acc_log=[]
    baseline_log=[]
    
    path='/content/gdrive/MyDrive/AI/weight/H'+str(i)+'/checkpoint.pth'
    path_log='/content/gdrive/MyDrive/AI/weight/H'+str(i)
    path_old='/content/gdrive/MyDrive/AI/weight/H'+str(i-1)
    torch.save(checkpoint, path)

    '''
    with open(path_old+'baseline_log.pkl', 'rb') as f:
        baseline_log = pickle.load(f)
    with open(path_old+'loss_log.pkl', 'rb') as f:
        loss_log = pickle.load(f)
    with open(path_old+'acc_log.pkl', 'rb') as f:
        acc_log = pickle.load(f)        
    '''
    for k in train_losses :
        loss_log.append(k)
    acc_log = acc_log + metrics
    baseline_log = baseline_log + baseline

    with open(path_log+'/baseline_log.pkl', 'wb') as f:
        pickle.dump(baseline_log, f)
    with open(path_log+'/loss_log.pkl', 'wb') as f:
        pickle.dump(loss_log, f)
    with open(path_log+'/acc_log.pkl', 'wb') as f:
        pickle.dump(acc_log, f)

In [ ]:
def Acc_fn(pred, targets):
  correct = 1*(pred == targets)
  return correct.mean().item()

In [ ]:
def train_one_epoch(model, data_loader, loss_fn, optimiser, device,i):
  train_losses=[]
  metrics=[]
  baseline=[]
  for inputs, targets in data_loader :
    inputs, targets = inputs.to(device), targets.to(device)
    #อันนี้เขา tab ลงไปอันนึง ทำไม?
    #calculate loss
    pred = model(inputs).argmax(dim=1)
    loss = loss_fn(pred, targets)
    print(pred.size(),targets.size())
    acc = Acc_fn(pred, targets)
    #backpropagate + update weight
    optimiser.zero_grad() #ลบ gradiun จาก batch ก่อนๆ
    loss.backward()
    optimiser.step()

  val_batch_loss = []
  for inputs, targets in data_loader :
      inputs, targets = inputs.to(device), targets.to(device)
      
      with torch.no_grad():
          preds = model(inputs)
          loss = loss_fn(preds, targets)
          train_losses.append(loss.detach().cpu())
          metrics.append(acc)
          baseline.append(loss_fn(torch.zeros_like(targets), targets))
  save(train_losses,metrics,baseline,i)
  print(f"Loss : {loss.item()}") #\t Acc:{acc}")

def train(model, data_loader, loss_fn, optimiser, device, epoch):
  for i in range(epoch):
    print(f"Epoch : {i+1}")
    train_one_epoch(model, data_loader, loss_fn, optimiser, device,i)
    print("---------------------------")
  print("Finished training")

train(model, train_dataloader, loss_fn, optimiser, device, 3)

Epoch : 1


RuntimeError: ignored

In [ ]:
for inputs, targets in train_dataloader :
  inputs, targets = inputs.to(device), targets.to(device)

  pred = model(inputs)
  pred = pred.argmax(dim=1)

torch.Size([128, 2560])
torch.Size([128])
tensor([ 309, 1450, 1447, 1445, 1449, 1447, 1447, 1445, 1447, 1925, 1927,  493,
         569,  495,  491,  570, 1925,  495, 1929, 1925, 1651, 1929, 1922,  486,
         487,  485, 1921,  486,  561,  485, 1921, 1921,  485, 1921,  562,  487,
        1922,  487,  485,  566, 1930, 1920,  566,  245,  163,  487,  566,  561,
        1921,  487, 1921,  313,  486, 1927, 1926, 1930, 1928,  829,  490, 1930,
         490,  253,  490,  490, 1645,  251,  165, 1933, 1933,  498,  498, 1459,
        1935, 1458,   74,  497,  497, 1458, 1933, 1934,  498,  497, 1933, 1457,
        1933, 1933, 1934, 1935, 1933, 1458,  498, 1935,  498, 1932,  303,  245,
         485,  303, 1925, 1642,  489,  489, 1925,  489, 1925, 1926,  485, 1457,
        1445, 1924,  485, 1925, 1925, 1447,  565,  167, 1924,  569,  250,  250,
         565,  250,  486,  486,  486,  485,  485,  486])
torch.Size([128, 2560])
torch.Size([128])
tensor([ 486,  486,  486,  486,  486,  486,  486,  486,  48

KeyboardInterrupt: ignored

In [ ]:
batch_size = 128
epochs = 10
lr = 0.001
device = 'cuda' if torch.cuda.is_available() else 'cpu'
loss_fn = nn.CrossEntropyLoss()
model = CNNNetwork()
optimiser = torch.optim.Adam(model.parameters(), lr=lr)
train_dataloader = DataLoader(usd, batch_size=batch_size)

In [ ]:
train(model, train_dataloader, loss_fn, optimiser, device, 3)

Epoch : 1
tensor([[0.0000e+00, 0.0000e+00, 9.4366e-04,  ..., 4.9695e-01, 5.8110e-01,
         4.2502e-02],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.3095e-02, 3.2366e-02,
         4.2334e-02],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 3.3291e-02, 3.1234e-02,
         4.2536e-02],
        ...,
        [4.4195e+00, 0.0000e+00, 0.0000e+00,  ..., 2.2199e-02, 2.3691e-02,
         5.2695e-02],
        [8.6738e+00, 0.0000e+00, 0.0000e+00,  ..., 2.3284e-02, 2.8728e-02,
         4.1282e-02],
        [2.8935e+00, 0.0000e+00, 0.0000e+00,  ..., 3.1893e-02, 3.1058e-02,
         4.6993e-02]], grad_fn=<ReshapeAliasBackward0>)


RuntimeError: ignored